In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import os
import random
from torch.autograd import Variable
import copy
from torch import nn, optim
import torch.optim as optim
import numpy as np
from torch.autograd import Variable
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import csv
import time


In [33]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Pytorch
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
fix_seed(SEED)

In [34]:
class Argments():
  def __init__(self):
    self.batch_size = 20
    self.test_batch = 1000
    self.global_epochs = 300
    self.local_epochs = 2
    self.lr = None
    self.momentum = 0.9
    self.weight_decay = 10**-4.0
    self.clip = 20.0
    self.partience = 300
    self.worker_num = 20
    self.sample_num = 20
    self.device = device = torch.device('cuda:0'if torch.cuda.is_available() else'cpu')
    self.criterion = nn.CrossEntropyLoss()

args = Argments()

In [35]:
lr = 0

In [37]:
lr_list = []
lr_list.append(10**-3.0)
lr_list.append(10**-2.5)
lr_list.append(10**-2.0)
lr_list.append(10**-1.5)
lr_list.append(10**-1.0)
lr_list.append(10**-0.5)
lr_list.append(10**0.0)
lr_list.append(10**0.5)

args.lr = lr_list[lr]

In [40]:
class LocalDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,worker_id):
    self.data = []
    self.target = []
    self.id = worker_id
    for data in dataset.take(len(dataset)):
      self.data.append(torch.tensor([data['pixels'].numpy()]))
      self.target.append(torch.tensor(data['label'].numpy().astype(np.int64)))

  def __getitem__(self, index):
    return self.data[index],self.target[index]

  def __len__(self):
    return len(self.data)

In [41]:
with open('../data/federated_trainset_femnist.pickle', 'rb') as f:
    all_federated_trainset = pickle.load(f)
with open('../data/federated_testset_femnist.pickle', 'rb') as f:
    all_federated_testset = pickle.load(f)
all_worker_num = len(all_federated_trainset)

In [42]:
worker_id_list = random.sample(range(all_worker_num),args.worker_num)
print(worker_id_list)
federated_trainset = []
federated_testset = []
for i in worker_id_list:
    federated_trainset.append(all_federated_trainset[i])
    federated_testset.append(all_federated_testset[i])

[2619, 456, 102, 3037, 1126, 1003, 914, 571, 3016, 419, 2771, 3033, 2233, 356, 2418, 1728, 130, 122, 383, 895]


In [43]:
federated_valset = [None]*args.worker_num
for i in range(args.worker_num):
  n_samples = len(federated_trainset[i])
  if n_samples==1:
    federated_valset[i] = copy.deepcopy(federated_trainset[i])
  else:
    train_size = int(len(federated_trainset[i]) * 0.7) 
    val_size = n_samples - train_size 
    federated_trainset[i],federated_valset[i] = torch.utils.data.random_split(federated_trainset[i], [train_size, val_size])

In [45]:
class CNN1(torch.nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(5408, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN2(torch.nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(9216, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

class CNN3(torch.nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(15488, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x
    
class CNN4(torch.nn.Module):
    def __init__(self):
        super(CNN4, self).__init__()
        self.conv2d_1 = torch.nn.Conv2d(1, 32, kernel_size=3)
        self.max_pooling = nn.MaxPool2d(2, stride=2)
        self.conv2d_2 = torch.nn.Conv2d(32, 64, kernel_size=3)
        self.conv2d_3 = torch.nn.Conv2d(64, 128, kernel_size=3)
        self.conv2d_4 = torch.nn.Conv2d(128, 256, kernel_size=3)
        self.dropout_1 = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.linear_1 = nn.Linear(25600, 128)
        self.dropout_2 = nn.Dropout(0.5)
        self.linear_2 = nn.Linear(128,62)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.conv2d_2(x)
        x = self.conv2d_3(x)
        x = self.conv2d_4(x)
        x = self.max_pooling(x)
        x = self.dropout_1(x)
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout_2(x)
        x = self.linear_2(x)
        return x

In [46]:
class Server():
  def __init__(self):
    self.model = CNN2()

  def create_worker(self,federated_trainset,federated_valset,federated_testset):
    workers = []
    for i in range(args.worker_num):
      workers.append(Worker(federated_trainset[i],federated_valset[i],federated_testset[i]))
    return workers

  def sample_worker(self,workers):
    sample_worker = []
    sample_worker_num = random.sample(range(args.worker_num),args.sample_num)
    for i in sample_worker_num:
      sample_worker.append(workers[i])
    return sample_worker


  def send_model(self,workers):
    nums = 0
    for worker in workers:
      nums += worker.train_data_num

    for worker in workers:
      worker.aggregation_weight = 1.0*worker.train_data_num/nums
      worker.model = copy.deepcopy(self.model)
      worker.model = worker.model.to(args.device)

  def aggregate_model(self,workers):   
    new_params = OrderedDict()
    for i,worker in enumerate(workers):
      worker_state = worker.model.state_dict()
      for key in worker_state.keys():
        if i==0:
          new_params[key] = worker_state[key]*worker.aggregation_weight
        else:
          new_params[key] += worker_state[key]*worker.aggregation_weight
      worker.model = worker.model.to('cpu')
      del worker.model
    self.model.load_state_dict(new_params)

In [47]:
class Worker():
  def __init__(self,trainset,valset,testset):
    self.trainloader = torch.utils.data.DataLoader(trainset,batch_size=args.batch_size,shuffle=True,num_workers=2)
    self.valloader = torch.utils.data.DataLoader(valset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.testloader = torch.utils.data.DataLoader(testset,batch_size=args.test_batch,shuffle=False,num_workers=2)
    self.model = None
    self.train_data_num = len(trainset)
    self.test_data_num = len(testset)
    self.aggregation_weight = None

  def local_train(self):
    acc_train,loss_train = train(self.model,args.criterion,self.trainloader,args.local_epochs)
    acc_valid,loss_valid = test(self.model,args.criterion,self.valloader)
    return acc_train,loss_train,acc_valid,loss_valid

    

In [48]:
def train(model,criterion,trainloader,epochs):
  optimizer = optim.SGD(model.parameters(),lr=args.lr,momentum=args.momentum,weight_decay=args.weight_decay)
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    count = 0
    for (data,labels) in trainloader:
      data,labels = Variable(data),Variable(labels)
      data,labels = data.to(args.device),labels.to(args.device)
      optimizer.zero_grad()
      outputs = model(data)
      loss = criterion(outputs,labels)
      running_loss += loss.item()
      predicted = torch.argmax(outputs,dim=1)
      correct += (predicted==labels).sum().item()
      count += len(labels)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
      optimizer.step()

  return 100.0*correct/count,running_loss/len(trainloader)



In [49]:
def test(model,criterion,testloader):
  model.eval()
  running_loss = 0.0
  correct = 0
  count = 0
  for (data,labels) in testloader:
    data,labels = data.to(args.device),labels.to(args.device)
    outputs = model(data)
    running_loss += criterion(outputs,labels).item()
    predicted = torch.argmax(outputs,dim=1)
    correct += (predicted==labels).sum().item()
    count += len(labels)

  accuracy = 100.0*correct/count
  loss = running_loss/len(testloader)


  return accuracy,loss

In [50]:
class Early_Stopping():
  def __init__(self,partience):
    self.step = 0
    self.loss = float('inf')
    self.partience = partience

  def validate(self,loss):
    if self.loss<loss:
      self.step += 1
      if self.step>self.partience:
        return True
    else:
      self.step = 0
      self.loss = loss

    return False

In [51]:
server = Server()
workers = server.create_worker(federated_trainset,federated_valset,federated_testset)
acc_train = []
loss_train = []
acc_valid = []
loss_valid = []

early_stopping = Early_Stopping(args.partience)

start = time.time()

for epoch in range(args.global_epochs):
  sample_worker = server.sample_worker(workers)
  server.send_model(sample_worker)

  acc_train_avg = 0.0
  loss_train_avg = 0.0
  acc_valid_avg = 0.0
  loss_valid_avg = 0.0
  for worker in sample_worker:
    acc_train_tmp,loss_train_tmp,acc_valid_tmp,loss_valid_tmp = worker.local_train()
    acc_train_avg += acc_train_tmp/len(sample_worker)
    loss_train_avg += loss_train_tmp/len(sample_worker)
    acc_valid_avg += acc_valid_tmp/len(sample_worker)
    loss_valid_avg += loss_valid_tmp/len(sample_worker)
  server.aggregate_model(sample_worker)
  '''
  server.model.to(args.device)
  for worker in workers:
    acc_valid_tmp,loss_valid_tmp = test(server.model,args.criterion,worker.valloader)
    acc_valid_avg += acc_valid_tmp/len(workers)
    loss_valid_avg += loss_valid_tmp/len(workers)
  server.model.to('cpu')
  '''
  print('Epoch{}  loss:{}  accuracy:{}'.format(epoch+1,loss_valid_avg,acc_valid_avg))
  acc_train.append(acc_train_avg)
  loss_train.append(loss_train_avg)
  acc_valid.append(acc_valid_avg)
  loss_valid.append(loss_valid_avg)

  if early_stopping.validate(loss_valid_avg):
    print('Early Stop')
    break
    
end = time.time()

Epoch1  loss:3.991999185085297  accuracy:4.837568947141925
Epoch2  loss:3.896000647544861  accuracy:5.676406166527012
Epoch3  loss:3.817688703536987  accuracy:6.156123506871529
Epoch4  loss:3.7792619228363034  accuracy:5.842471396472004
Epoch5  loss:3.747075366973877  accuracy:6.165615146725727
Epoch6  loss:3.727268695831298  accuracy:6.187896153100055
Epoch7  loss:3.70428203344345  accuracy:6.026246578785733
Epoch8  loss:3.7050479054450998  accuracy:7.211237740364301
Epoch9  loss:3.6906812906265256  accuracy:6.962124614114873
Epoch10  loss:3.6829524874687194  accuracy:6.513399522460415
Epoch11  loss:3.685239803791046  accuracy:6.907495612064233
Epoch12  loss:3.683282315731049  accuracy:6.769938263543429
Epoch13  loss:3.6716830730438232  accuracy:7.0810850187895635
Epoch14  loss:3.6805065393447873  accuracy:7.715754803501799
Epoch15  loss:3.671628868579864  accuracy:6.246049109219492
Epoch16  loss:3.664927935600281  accuracy:7.210918471965497
Epoch17  loss:3.6609030842781065  accuracy:

Epoch136  loss:1.6265403747558593  accuracy:62.64216679319585
Epoch137  loss:1.6072122454643252  accuracy:62.138876018116484
Epoch138  loss:1.5986374497413636  accuracy:62.80107630651581
Epoch139  loss:1.5804342210292814  accuracy:63.31705246257577
Epoch140  loss:1.5740407764911648  accuracy:63.44010345774331
Epoch141  loss:1.5717752635478976  accuracy:63.265797552925854
Epoch142  loss:1.5469716489315033  accuracy:63.89711010754093
Epoch143  loss:1.544756144285202  accuracy:64.36679688417813
Epoch144  loss:1.5423519730567932  accuracy:63.91269258293166
Epoch145  loss:1.5411623418331148  accuracy:63.69094240278793
Epoch146  loss:1.5205180644989011  accuracy:63.753893334270884
Epoch147  loss:1.5059996485710145  accuracy:64.83133084691882
Epoch148  loss:1.499684923887253  accuracy:64.87217288619921
Epoch149  loss:1.4942011535167692  accuracy:64.14862909394087
Epoch150  loss:1.4899069160223006  accuracy:64.26220381743828
Epoch151  loss:1.4821888029575347  accuracy:64.10380452575383
Epoch15

Epoch269  loss:0.937289386987686  accuracy:74.05430409113039
Epoch270  loss:0.9349562019109725  accuracy:74.07491865981089
Epoch271  loss:0.9286262214183807  accuracy:74.0988442196101
Epoch272  loss:0.9240083515644073  accuracy:74.41952339776299
Epoch273  loss:0.930867812037468  accuracy:74.76974423448681
Epoch274  loss:0.9233167976140976  accuracy:74.57961207092569
Epoch275  loss:0.9190049469470978  accuracy:74.79653982108178
Epoch276  loss:0.9272795408964158  accuracy:74.31349132185868
Epoch277  loss:0.9274469763040541  accuracy:74.09418658697275
Epoch278  loss:0.9210182964801789  accuracy:75.06100681673992
Epoch279  loss:0.9140280783176423  accuracy:74.91652442870874
Epoch280  loss:0.9267109632492065  accuracy:74.42715497533453
Epoch281  loss:0.9161523461341858  accuracy:74.35520348683016
Epoch282  loss:0.9246771425008774  accuracy:75.0096971641778
Epoch283  loss:0.9105062812566758  accuracy:74.51596444504435
Epoch284  loss:0.9095922380685807  accuracy:75.02605021049109
Epoch285  lo

In [53]:
acc_test = []
loss_test = []

server.model.to(args.device)

nums = 0
for worker in workers:
  nums += worker.test_data_num

start = time.time()

for i,worker in enumerate(workers):
  worker.aggregation_weight = 1.0*worker.test_data_num/nums
  acc_tmp,loss_tmp = test(server.model,args.criterion,worker.testloader)
  acc_test.append(acc_tmp)
  loss_test.append(loss_tmp)
  print('Worker{} accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))

end = time.time()

acc_test_avg = sum(acc_test)/len(acc_test)
loss_test_avg = sum(loss_test)/len(loss_test)
print('Test  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 accuracy:70.58823529411765  loss:1.0342817306518555
Worker2 accuracy:85.71428571428571  loss:0.4366610646247864
Worker3 accuracy:72.5  loss:0.9161068797111511
Worker4 accuracy:66.66666666666667  loss:1.6200659275054932
Worker5 accuracy:82.3529411764706  loss:0.6161081790924072
Worker6 accuracy:76.19047619047619  loss:0.6697072386741638
Worker7 accuracy:73.6842105263158  loss:0.9483001232147217
Worker8 accuracy:78.125  loss:0.6947300434112549
Worker9 accuracy:31.57894736842105  loss:2.149538516998291
Worker10 accuracy:78.04878048780488  loss:0.7790355086326599
Worker11 accuracy:55.55555555555556  loss:1.6017330884933472
Worker12 accuracy:73.6842105263158  loss:0.7236959338188171
Worker13 accuracy:78.94736842105263  loss:0.7788953185081482
Worker14 accuracy:63.333333333333336  loss:1.7813154458999634
Worker15 accuracy:63.1578947368421  loss:0.7827916741371155
Worker16 accuracy:70.58823529411765  loss:0.767959713935852
Worker17 accuracy:69.6969696969697  loss:1.356797695159912
Wor

In [55]:
acc_tune_test = []
loss_tune_test = []
acc_tune_valid = []
loss_tune_valid = []

start = time.time()

for i,worker in enumerate(workers):
    worker.model = copy.deepcopy(server.model)
    worker.model = worker.model.to(args.device)
    _,_,acc_tmp,loss_tmp = worker.local_train()
    acc_tune_valid.append(acc_tmp)
    loss_tune_valid.append(loss_tmp)
    print('Worker{} Valid accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    
    acc_tmp,loss_tmp = test(worker.model,args.criterion,worker.testloader)
    acc_tune_test.append(acc_tmp)
    loss_tune_test.append(loss_tmp)
    print('Worker{} Test accuracy:{}  loss:{}'.format(i+1,acc_tmp,loss_tmp))
    worker.model = worker.model.to('cpu')
    del worker.model

end = time.time()

acc_valid_avg = sum(acc_tune_valid)/len(acc_tune_valid)
loss_valid_avg = sum(loss_tune_valid)/len(loss_tune_valid)
print('Validation(tune)  loss:{}  accuracy:{}'.format(loss_valid_avg,acc_valid_avg))
acc_test_avg = sum(acc_tune_test)/len(acc_tune_test)
loss_test_avg = sum(loss_tune_test)/len(loss_tune_test)
print('Test(tune)  loss:{}  accuracy:{}'.format(loss_test_avg,acc_test_avg))

Worker1 Valid accuracy:74.4186046511628  loss:0.7241300344467163
Worker1 Test accuracy:82.3529411764706  loss:0.9369863867759705
Worker2 Valid accuracy:85.1063829787234  loss:0.589558482170105
Worker2 Test accuracy:85.71428571428571  loss:0.3438548743724823
Worker3 Valid accuracy:69.52380952380952  loss:0.944478452205658
Worker3 Test accuracy:82.5  loss:0.8113101124763489
Worker4 Valid accuracy:76.08695652173913  loss:0.8718819618225098
Worker4 Test accuracy:72.22222222222223  loss:1.3708428144454956
Worker5 Valid accuracy:76.92307692307692  loss:0.692795991897583
Worker5 Test accuracy:88.23529411764706  loss:0.47469061613082886
Worker6 Valid accuracy:72.72727272727273  loss:0.9787224531173706
Worker6 Test accuracy:71.42857142857143  loss:0.7181011438369751
Worker7 Valid accuracy:81.0  loss:0.7959750890731812
Worker7 Test accuracy:71.05263157894737  loss:0.9811251759529114
Worker8 Valid accuracy:70.58823529411765  loss:0.9337114095687866
Worker8 Test accuracy:75.0  loss:0.6661114096641